In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/BIMAG

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/BIMAG


In [ ]:
!pip install leidenalg
!pip install umap_learn
!pip install umap-learn[plot]
!pip install tqdm_joblib
!pip install resample
!pip install clusim
!pip install anndata
!pip install scanpy
!pip install fastcluster

In [ ]:
import numpy as np
import pandas as pd
import torch
from scLENS import scLENS

import cProfile, pstats
import umap
import umap.plot

from sklearn.metrics import rand_score, normalized_mutual_info_score
from clusim.clustering import Clustering
import clusim.sim as sim

import matplotlib.pyplot as plt
import seaborn as sns

import re
from scLENS.clustering import scSHC, find_clusters
import anndata
import scanpy as sc

# Params

In [ ]:
  # Method-agnostic hyperparameters
params = {'resolutions': np.arange(0.1, 2, 0.1), #np.arange(0.05, 2, 0.05), # [0.3, 0.5, 0.8, 1, 1.2, 1.6, 2, 4, 6, 8],
          'reps': 10,
          'n_jobs': -1}

In [ ]:
colors = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray', 'tab:olive', 'tab:cyan', 'k']

# Display Helper

In [ ]:
def display_mappings(X, clusterings, names):
  mapper = umap.UMAP(metric='cosine').fit_transform(X)

  fig, axs = plt.subplots(1, len(clusterings), figsize=(40, 10))
  for ax, cls in enumerate(clusterings):
    labels = np.unique(cls)
    for i, l in enumerate(labels):
      idx = np.where(cls == l)
      axs[ax].scatter(mapper[idx, 0], mapper[idx, 1], s=2, label=l)

    axs[ax].legend()
    axs[ax].set_xlabel('UMAP1')
    axs[ax].set_ylabel('UMAP2')
    axs[ax].tick_params(axis='both',
                  which='both',
                  bottom=False,
                  top=False,
                  labelbottom=False,
                  labelleft=False)
    axs[ax].set_title(names[ax])
  plt.show()

# Evaluate Func

In [ ]:
def evaluate(filepaths, save_scl, save_noscl, overwrite=False, **params):
  scl_pool, noscl_pool = [], []

  for file in filepaths:
    if overwrite or save.loc[file].isnull().all():
      print(file)
      df = pd.read_csv(file)
      y_true_full = df['cell']
      df = df.drop('cell', axis=1)

      sclens = scLENS(device=torch.device('cuda:0'))
      sclens.preprocess(df)
      X_transform = sclens.fit_transform()

      adata = anndata.AnnData(X=df, obs=pd.DataFrame(y_true_full))

      sc.pp.filter_cells(adata, min_genes=200)
      sc.pp.filter_genes(adata, min_cells=3)

      sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

      kept_cells = adata.obs.n_genes_by_counts.values < 2500
      adata = adata[adata.obs.n_genes_by_counts < 2500, :]

      sc.pp.normalize_total(adata, target_sum=1e4)
      sc.pp.log1p(adata)
      sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
      adata = adata[:, adata.var.highly_variable]
      sc.pp.scale(adata, max_value=10)

      sc.tl.pca(adata, svd_solver='arpack')

      X = df.values[kept_cells]
      X_transform = adata.obsm['X_pca']
      y_true = adata.obs['cell']

      sch_scl = sclens.cluster(df, method='scSHC')
      sch_noscl = scSHC(X, X_transform)

      sch_full = np.zeros_like(y_true_full) - 1
      sch_full[kept_cells] = sch_noscl

      sch_cls_noscl = Clustering().from_membership_list(sch_full)
      sch_cls_scl = Clustering().from_membership_list(sch_scl)
      true_cls = Clustering().from_membership_list(y_true_full)

      # display_mappings(X_transform, [y_true, sch_noscl, sch_scl], ['True', 'scSHC', 'scLENS + scSHC'])

      noscl_score = sim.element_sim(true_cls, sch_cls_noscl)
      scl_score = sim.element_sim(true_cls, sch_cls_scl)

      scl_pool.append(scl_score)
      noscl_pool.append(noscl_score)

      scores = {'scSHC': noscl_score, 'scLENS + scSHC': scl_score}
      schemes = ['Element-Centric Clustering']

      x = np.arange(len(schemes))
      width = 0.25
      multiplier = 0

      fig, ax = plt.subplots(layout='constrained')

      for name, val in scores.items():
          offset = width * multiplier
          rects = ax.bar(x + offset, val, width, label=name)
          ax.bar_label(rects, padding=2)
          multiplier += 1

      ax.set_ylabel('Score')
      ax.set_title(' '.join(file.split('/')[2:]))
      ax.set_xticks(x + width/2, schemes)
      ax.legend(loc='upper right', ncols=2)

      plt.show()
      del sclens
      torch.cuda.empty_cache()

      save_scl.loc[file, 'ECC'] = scl_score
      save_noscl.loc[file, 'ECC'] = noscl_score
      save_scl.to_csv('scSHC_scores.csv')
      save_noscl.to_csv('scSHC_nosclens_scores.csv')

  return scl_pool, noscl_pool

# Run Dataset Evaluation

In [ ]:
import os

filepaths = []

for dir in os.listdir('scLENS/data'):
  if dir.endswith('.csv.gz'):
    filepaths.append(os.path.join('scLENS/data', dir))
  else:
    for dir2 in os.listdir(os.path.join('scLENS/data', dir)):
      if dir2.endswith('.csv.gz'):
        filepaths.append(os.path.join('scLENS/data', dir, dir2))
      else:
        for f in os.listdir(os.path.join('scLENS/data', dir, dir2)):
          if f.endswith('.csv.gz'):
            filepaths.append(os.path.join('scLENS/data', dir, dir2, f))

filepaths

['scLENS/data/Z8eq.csv.gz',
 'scLENS/data/sim_T_muris/balanced/T_muris_1019.csv.gz',
 'scLENS/data/sim_T_muris/balanced/T_muris_2006.csv.gz',
 'scLENS/data/sim_T_muris/balanced/T_muris_3016.csv.gz',
 'scLENS/data/sim_T_muris/balanced/T_muris_3966.csv.gz',
 'scLENS/data/sim_T_muris/balanced/T_muris_5033.csv.gz',
 'scLENS/data/sim_T_muris/imbalanced/tmp_1.csv.gz',
 'scLENS/data/sim_T_muris/imbalanced/tmp_2.csv.gz',
 'scLENS/data/sim_T_muris/imbalanced/tmp_3.csv.gz',
 'scLENS/data/sim_T_muris/imbalanced/tmp_4.csv.gz',
 'scLENS/data/sim_T_muris/imbalanced/tmp_5.csv.gz',
 'scLENS/data/sim_Tcell/2250-1220.csv.gz',
 'scLENS/data/sim_Tcell/2251-1179.csv.gz',
 'scLENS/data/sim_Tcell/2972-1523.csv.gz',
 'scLENS/data/sim_Tcell/3276-1042.csv.gz',
 'scLENS/data/sim_Tcell/3689-1878.csv.gz',
 'scLENS/data/sim_Tcell/4249-2916.csv.gz',
 'scLENS/data/sim_Tcell/4320-2134.csv.gz',
 'scLENS/data/sim_Tcell/4797-2332.csv.gz',
 'scLENS/data/sim_Tcell/5930-3210.csv.gz',
 'scLENS/data/sim_Tcell/5960-1400.csv.gz

In [ ]:
if not os.path.exists('scSHC_scores.csv'):
  save = pd.DataFrame(index=filepaths, columns=['ECC'])
  save.to_csv('scSHC_scores.csv')
else:
  save = pd.read_csv('scSHC_scores.csv', index_col=0)

if not os.path.exists('scSHC_nosclens_scores.csv'):
  save_noscl = pd.DataFrame(index=filepaths, columns=['ECC'])
  save_noscl.to_csv('scSHC_nosclens_scores.csv')
else:
  save_noscl = pd.read_csv('scSHC_nosclens_scores.csv', index_col=0)

In [ ]:
save

,ECC
scLENS/data/Z8eq.csv.gz,0.631266
scLENS/data/sim_T_muris/balanced/T_muris_1019.csv.gz,0.744875
scLENS/data/sim_T_muris/balanced/T_muris_2006.csv.gz,0.225107
scLENS/data/sim_T_muris/balanced/T_muris_3016.csv.gz,0.429963
scLENS/data/sim_T_muris/balanced/T_muris_3966.csv.gz,0.406115
scLENS/data/sim_T_muris/balanced/T_muris_5033.csv.gz,0.666753
scLENS/data/sim_T_muris/imbalanced/tmp_1.csv.gz,0.253664
scLENS/data/sim_T_muris/imbalanced/tmp_2.csv.gz,0.476338
scLENS/data/sim_T_muris/imbalanced/tmp_3.csv.gz,0.688160
scLENS/data/sim_T_muris/imbalanced/tmp_4.csv.gz,0.706244


In [ ]:
chsr_score, mltk_score = evaluate(filepaths, save, save_noscl, overwrite=False, **params)

scLENS/data/Zheng_real_data/Balanced/z_data_12073.csv.gz


  0%|          | 0/10 [00:00<?, ?it/s]

/content/drive/MyDrive/BIMAG/scLENS/scLENS.py:345: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_clean = torch.tensor(X).to(self.device, dtype=torch.double)
/usr/local/lib/python3.10/dist-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_scale.py:299: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Fitting model
Making PD
Generating null stats
ClusterID: 23940, Num tests: 11971, Significance: 3.982529019251016e-78


/content/drive/MyDrive/BIMAG/scLENS/cluster_utils.py:339: RuntimeWarning: invalid value encountered in divide
  y2 = (X - mu) ** 2 / mu


Fitting model
Making PD
Generating null stats
ClusterID: 23939, Num tests: 4564, Significance: 0.0
Fitting model
Making PD
Generating null stats
ClusterID: 23937, Num tests: 2943, Significance: 0.0
Fitting model
Making PD
Generating null stats
ClusterID: 23932, Num tests: 1579, Significance: 0.0
Fitting model
Making PD
Generating null stats
ClusterID: 23930, Num tests: 109, Significance: 0.637950287578165
Fitting model
Making PD
Generating null stats
ClusterID: 23911, Num tests: 1470, Significance: 1.5013668945333427e-05
Fitting model
Making PD
Generating null stats
ClusterID: 23908, Num tests: 1466, Significance: 0.7965458544241197


/content/drive/MyDrive/BIMAG/scLENS/cluster_utils.py:454: RuntimeWarning: divide by zero encountered in divide
  X = np.transpose(X.T / l2_norm) * np.mean(l2_norm)
/content/drive/MyDrive/BIMAG/scLENS/cluster_utils.py:454: RuntimeWarning: invalid value encountered in divide
  X = np.transpose(X.T / l2_norm) * np.mean(l2_norm)


LinAlgError: Eigenvalues did not converge

In [ ]:
save.mean()

# Load Scores, Compare

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/BIMAG

In [ ]:
save_ecc = pd.read_csv('save.csv', index_col=0)
save_ecc

In [ ]:
save_rand_nmi = pd.read_csv('RandIndex_NMI.csv', index_col=0)
save_rand_nmi

In [ ]:
scl_scores = pd.concat([save_ecc, save_rand_nmi], axis=1)
scl_scores = scl_scores.drop(['ChooseR_ECC', 'ChooseR_RandIndex', 'ChooseR_NormMI'], axis=1)
scl_scores

In [ ]:
scl_scores.mean()

In [ ]:
noscl_scores = pd.read_csv('save_no_sclens.csv', index_col=0)
noscl_scores

In [ ]:
scl_means = scl_scores.mean()
noscl_means = noscl_scores.mean()

scores = {'scLENS + MultiK': [scl_means['MultiK_RandIndex'], scl_means['MultiK_NormMI'], scl_means['MultiK_ECC']],
          'MultiK': [noscl_means['MultiK_RandIndex'], noscl_means['MultiK_NormMI'], noscl_means['MultiK_ECC']]}
schemes = ['Rand Index', 'Norm Mutual Information', 'Element-Centric']

x = np.arange(len(schemes))
width = 0.25
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for name, val in scores.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, val, width, label=name)
    ax.bar_label(rects, padding=2)
    multiplier += 1

ax.set_ylabel('Score')
ax.set_title('Mean Scores')
ax.set_xticks(x + width/2, schemes)
ax.legend(loc='upper right', ncols=2)

plt.show()

# Per Dataset

In [ ]:
fig = plt.figure(figsize=(5, 10))
ax = fig.add_subplot(111)

for dataset in scl_scores.index:
  scl = scl_scores.loc[dataset, 'MultiK_ECC']
  noscl = noscl_scores.loc[dataset, 'MultiK_ECC']

  ax.plot([0, 1], [noscl, scl], 'k--', linewidth=1)
  ax.scatter(0, noscl, color='k')
  ax.scatter(1, scl, color='k')

ax.set_title('Per-Dataset ECC')
ax.set_xticks([0, 1])
ax.set_xticklabels(['No scLENS', 'scLENS'])
ax.set_ylabel('ECC Score')

plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 10))
ax = fig.add_subplot(111)

for dataset in scl_scores.index:
  scl = scl_scores.loc[dataset, 'MultiK_RandIndex']
  noscl = noscl_scores.loc[dataset, 'MultiK_RandIndex']

  ax.plot([0, 1], [noscl, scl], 'k--', linewidth=1)
  ax.scatter(0, noscl, color='k')
  ax.scatter(1, scl, color='k')

ax.set_title('Per-Dataset Rand Index')
ax.set_xticks([0, 1])
ax.set_xticklabels(['No scLENS', 'scLENS'])
ax.set_ylabel('Rand Index Score')

plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 10))
ax = fig.add_subplot(111)

for dataset in scl_scores.index:
  scl = scl_scores.loc[dataset, 'MultiK_NormMI']
  noscl = noscl_scores.loc[dataset, 'MultiK_NormMI']

  ax.plot([0, 1], [noscl, scl], 'k--', linewidth=1)
  ax.scatter(0, noscl, color='k')
  ax.scatter(1, scl, color='k')

ax.set_title('Per-Dataset NMI')
ax.set_xticks([0, 1])
ax.set_xticklabels(['No scLENS', 'scLENS'])
ax.set_ylabel('Norm Mutual Information Score')

plt.show()

# Investigate

In [ ]:
from scLENS.clustering import multiK

In [ ]:
scl_check = scl_scores.drop(['MultiK_RandIndex', 'MultiK_NormMI'], axis=1)[scl_scores['MultiK_ECC'] < noscl_scores['MultiK_ECC']]
noscl_check = noscl_scores.drop(['MultiK_RandIndex', 'MultiK_NormMI'], axis=1)[scl_scores['MultiK_ECC'] < noscl_scores['MultiK_ECC']]
scl_check

In [ ]:
noscl_check

In [ ]:
for dataset in scl_check.index:
  df = pd.read_csv(dataset)
  y_true = df['cell']
  df = df.drop(['cell'], axis=1)

  scl = scLENS(device=torch.device('cuda:0'))
  scl.preprocess(df)
  X_transform_scl = scl.fit_transform()
  mltk_scl = scl.cluster(df, method='multiK', **params)

  adata = anndata.AnnData(X=df, obs=pd.DataFrame(y_true))

  sc.pp.filter_cells(adata, min_genes=200)
  sc.pp.filter_genes(adata, min_cells=3)

  sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

  adata = adata[adata.obs.n_genes_by_counts < 2500, :]

  sc.pp.normalize_total(adata, target_sum=1e4)
  sc.pp.log1p(adata)
  sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
  adata = adata[:, adata.var.highly_variable]
  sc.pp.scale(adata, max_value=10)

  sc.tl.pca(adata, svd_solver='arpack')

  X_transform_noscl = adata.obsm['X_pca']
  y_true_noscl = adata.obs['cell']

  res = multiK(X_transform_noscl, **params)

  mltk_noscl = find_clusters(X_transform_noscl,
                             n_neighbors=20,
                             min_weight=1/15,
                             res=res,
                             n_iterations=-1)

  display_mappings(X_transform_scl, [y_true, mltk_scl], ['True', 'scLENS MultiK'])
  display_mappings(X_transform_noscl, [y_true_noscl, mltk_noscl], ['True', 'No scLENS MultiK'])